# Spare-it Proof of Concept (PoC)

### Objective: 
estimate automatically composition of waste inside bins based on itms visible on images

### High-Level Goal: 
count multiple labels of waste (classification); use open source

### Steps: 
1. count instances of each mterial type in each image
2. apply sorting rules to material types based on type of bin (compost/mixed recycling/trash)

### Result: 
1. compute composition of waste (% of items of each material type)
2. the waste contamination rate, which is the proportion of items thave have been mistakingly put in a compost or recycling bin,  over the total number of visible objects.
3. the missed opportunities rate, which is the proportion of items that could have been recycled or composted in a trash bin,  over the total number of visible objects.

In [ ]:
pip install torch ultralytics

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

In [ ]:
import torch
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

import os
import cv2
import json
from matplotlib import pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image

In [ ]:
# Download COCO val
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

In [ ]:
!yolo train data=dataset.yaml model=yolov8s.yaml epochs=50 batch=16 imgsz=640

In [ ]:
# Path to your image
folder_path = '/Users/tessa/Desktop/spare-it-images' 

# Supported image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# List to store loaded images
images = []

# get all list of files in the path
files = os.listdir(folder_path)
print(f"Total number of files in the directory: {len(files)}")

# print number of json files
json_files = [file for file in files if file.endswith('.json')]
print(f"Number of .json files in the folder: {len(json_files)}")

# print number of image files
image_extensions = ['.jpg', '.jpeg', '.png', '.gif']
image_files = [file for file in files if any(file.endswith(ext) for ext in image_extensions)]
print(f"Number of image files in the folder: {len(image_files)}")

# list out all other files extension
other_files = [file for file in files if file not in json_files + image_files]
print(f"Files not classified as JSON or image files: {other_files}")

In [ ]:
model = YOLO("yolov8n")

In [ ]:
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# Find all images with a corresponding JSON file
available_images = []
for file in os.listdir(folder_path):
    name, ext = os.path.splitext(file)
    if ext.lower() in image_extensions and os.path.exists(os.path.join(folder_path, name + '.json')):
        available_images.append(file)

print(f"Found {len(available_images)} images with corresponding JSON files.")

In [ ]:
def load_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    annotations = data['annotations']
    # Process annotations as needed, e.g., extracting bounding boxes or categories
    # This is an example and may need to be adapted
    for ann in annotations:
        bbox = ann['bbox']  # Example: [x, y, width, height]
        category_id = ann['category_id']
        # Lookup category name (example)
        category_name = next((cat['name'] for cat in data['categories'] if cat['id'] == category_id), None)
        print(f"Annotation: {category_name}, BBox: {bbox}")

# Example usage for the first image with a corresponding JSON file
if available_images:
    first_image_json_path = os.path.join(folder_path, os.path.splitext(available_images[0])[0] + '.json')
    load_annotations(first_image_json_path)


In [ ]:
import os

# Directory containing your images
image_dir = '/Users/tessa/Desktop/spare-it-images'  # Update this path

# Supported image formats
image_formats = ('.bmp', '.dng', '.jpeg', '.jpg', '.mpo', '.png', '.tif', '.tiff', '.webp')

# Iterate through each file in the directory
for file_name in os.listdir(image_dir):
    # Check if the file is an image based on its extension
    if file_name.lower().endswith(image_formats):
        image_path = os.path.join(image_dir, file_name)  # Construct the full path to the image

        # Perform inference
        results = model(image_path)

        # Inspect the type and structure of results
        print(f"Processing {file_name}:")
        print(type(results))
        print(len(results))
        if len(results) > 0:
            print(type(results[0]))
            print(results[0])
    else:
        print(f"Skipping {file_name}: Not an image file.")


In [ ]:
# Perform inference
results = model(image_path)

# Inspect the type and structure of results
print(type(results))
print(len(results))
if len(results) > 0:
    print(type(results[0]))
    print(results[0])

In [ ]:
# Assuming detections is obtained correctly from 'results'
if hasattr(results, 'boxes') and results.boxes:
    detections = results.boxes.xyxy[0]  # Access detections assuming they're stored in this attribute

    # Debugging: Print the structure of the first detection
    if len(detections) > 0:
        print("Detection structure:", detections[0].shape)
        print("Detection content:", detections[0])


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

# Directory containing images
directory_path = '/Users/tessa/Desktop/spare-it-images'

# Supported image formats
image_formats = ('.bmp', '.dng', '.jpeg', '.jpg', '.mpo', '.png', '.tif', '.tiff', '.webp')

# Loop through files in the directory
for file_name in os.listdir(directory_path):
    # Construct the full file path
    image_path = os.path.join(directory_path, file_name)
    
    # Check if the path is an actual file and has a supported image format
    if os.path.isfile(image_path) and file_name.lower().endswith(image_formats):
        # Open and visualize the image
        img = Image.open(image_path)
        fig, ax = plt.subplots(1)
        ax.imshow(img)
        plt.axis('off')  # Hides the axis
        plt.title(file_name)  # Optional: add the file name as title
        plt.show()


In [ ]:
# Assuming 'results' is the list containing your model's inference output
results_obj = results[0]  # Extracting the Results object assuming it's the first element

# Print the type and available attributes or methods of the results object
print(f"Type of results_obj: {type(results_obj)}")
print(dir(results_obj))  # This will list all attributes and methods

# If 'boxes' is an attribute, investigate its structure further
if hasattr(results_obj, 'boxes'):
    print("Found 'boxes' attribute.")
    print("Type of 'boxes':", type(results_obj.boxes))
    print("Content of 'boxes':", results_obj.boxes)

    # If 'boxes' contains tensor data, let's try to print its size
    if hasattr(results_obj.boxes, 'xyxy'):
        print("Found 'xyxy' attribute in 'boxes'.")
        print("Type of 'boxes.xyxy':", type(results_obj.boxes.xyxy))
        print("Number of detections:", len(results_obj.boxes.xyxy[0]))
        for det in results_obj.boxes.xyxy[0]:
            print(det)


The output above provides insight into the strcture of the results_obj and specifically the boxes attribute containing detection data. The boxes object within result_obj has an xyxy attribute which is a tensor containing the bounding box coordinates in the format[x1, y1, x2, y2], alongside other attributes such as cls for class IDs and conf for confidence scores.

From this, we can construct a visualization script that directly utilizes this structured data. Notably, the xyxy tensor's structure indicates each component (x1, y1, x2, y2) is individually listed when iterated, rather than as a collective set per detection, which is unusual. Normally, you would expect a single iteration to yield all components of a detection (i.e., a bounding box and possibly class and confidence) together. However, the provided output suggests we might need to access the complete tensor directly rather than iterating over its elements for individual bounding boxes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load the original image from the path
img = Image.open(results_obj.path)
fig, ax = plt.subplots(1)
ax.imshow(img)

# Assuming 'xyxy' is the attribute containing bounding box coordinates
bbox_tensor = results_obj.boxes.xyxy[0]  # Access the tensor directly

# The provided output suggests accessing the whole detection tensor at once
# Here we assume the tensor structure is [x1, y1, x2, y2, conf, cls_id] for each detection
if bbox_tensor.shape[0] > 0:  # Ensure there are detections
    # As per the output, let's directly use the tensor for plotting
    x1, y1, x2, y2 = bbox_tensor[0], bbox_tensor[1], bbox_tensor[2], bbox_tensor[3]
    conf = results_obj.boxes.conf  # Assuming this is how confidence is accessed
    cls_id = results_obj.boxes.cls  # Assuming this is how class ID is accessed
    
    # Draw the bounding box
    rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor='r', facecolor='none')
    ax.add_patch(rect)

    # Label with class name and confidence
    label = f"{results_obj.names[int(cls_id[0].item())]}: {conf[0].item():.2f}"
    ax.text(x1, y1, label, color='white', va='top', fontsize=8, bbox=dict(facecolor='red', alpha=0.5))

plt.axis('off')  # Hide axes ticks
plt.show()


### Key Adjustments:
This script assumes the results_obj.boxes.xyxy tensor directly provides the bounding box coordinates for the first detection. Given your output, it seems we might need to reconsider how the detections are iterated or accessed, as the structure provided does not clearly delineate multiple detections.

Access to conf and cls directly from results_obj.boxes is based on your structure insight, but might need adjustment if these attributes store multiple values or are structured differently.
This visualization script is tailored to the specific structure you've encountered and the output shared. It's important to adjust the script if the assumptions made here about accessing the conf and cls_id do not hold or if further clarification on the structure is provided. The unusual formatting of the detection output may require further adjustment or clarification from Ultralytics YOLO documentation or support resources to ensure accurate access and representation of the detection data.